
<img src="https://www.python.org/static/community_logos/python-logo-generic.svg" height="500" width="500"> 

# <center> Грабёж, разбой и другие незаконные деяния с помощью Python. <br> <br> Наше первое дело. Грабим ЦИАН.   </center>
---------


<img align="center" src="https://www.cian.ru/promo/adv/assets/images/logo.svg" height="300" width="300"> 




## Что мы хотим получить? 

Итак, нам нужно распарсить [CIAN.ru](http://cian.ru) и получить из него как минимум следующий список переменных по каждой квартирке из каждого округа Москвы

- **N** – номер квартиры по порядку
- **Rooms** – число комнат 
- **Price** – цена
- **Totsp** – общая площадь квартиры, кв. м.
- **Livesp** – жилая площадь квартиры, кв. м.
- **Kitsp** – площадь кухни, кв. м.
- **Dist** – расстояние от центра в км. (можно по Пифагору)
- **Metrdist** – расстояние до метро в минутах
- **Walk** – 1 – если пешком от метро, 0 – если на транспорте
- **Brick** – 1 – кирпичный/монолит/жб, 0 – другой
- **Tel** – 1 – если есть, 0 – нет
- **Bal** – 1 – есть балкон/лоджия, 0 – нет
- **Floor** – номер этажа, на котором расположена квартира
- **Nfloors** – всего этажей в доме
- **New** – 1 – первичный рынок, 0 - вторичка
- ** И другие** 
---


## Для чего всё это нужно? 



**Во-первых**, написать свой первый (второй, третий...N+1) парсер, познакомиться получше с питоном и некоторыми его библиотеками.

**Во-вторых**, получив такой датасетик можно будет заняться его обработкой, очисткой, визуализацией, и прочими радостями, которые обязательно сопровождают работу с данными

**В-третьих**, на основании обработанных данных можно строить модели! Например, объясняющие цену квартирки, что поможет понять, кому и где на Руси жить хорошо, или по поиску аномалий и выбросов, чтобы узнать, где у нас есть недооцененное/переоцененное жильё. 

**В-четвёртых**, от того, что происходит ниже иногда хочется кричать.


<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/2.png" height="600" width="600">


---


## 1. Пара зубодробительных определений

* **Парсер** – это скрипт, который грабит информацию с сайта
* **Краулер** – это часть парсера, которая бродит по ссылкам
* **Крайлинг** – это переход по страницам и ссылкам
* **Скрапинг** – это сбор данных со страниц
* **Парсинг** – это сразу и то, и другое! 

## 2. Что такое HTML? 

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тэгами**. Если быть совсем дотошным, то HTML является чпстным случаем некоего страшного стандарта SGML. Другим частным случаем является XML, с которым мы ещё встретимся. 

Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта. 

Можно увидеть, что HTML-страница это ни что иное как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `< ... >` открывает тег, а `</ ... >` закрывает его. Все, что находится между этими двумя командами подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и `</p>` – это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега есть свои потомки (дети) - те теги, которые вложены в него и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div> 
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий, жирный кусок
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    
<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/tree.png" height="500" width="500"> 

Можно работать с этим html как с тексом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

Вручную обходить эти деревья не очень приятно, поэтому есть специальные языки для обхода деревьев.

- CSS-селектор
- XPath
- Всякие разные библиотеки для всяких разных языков, например, BeautifulSoup для питона. 

## 3. Как работает интернет или сказ о Бобе и Алисе

Алиса --- это веб-сервер. Боб --- это обычный компьютер. Боб хочет подсоединиться к Алисе. Как это происходит: 

* Боб отправляет Алисе последовательности из нулей и единиц. 1 - высокое напряжение, 0 - низкое. Эти последовательности шифруют просьбу Боба подконектиться к Алисе.

* Маршрутиризатор Боба собирает все эти последовательности в пакет и отправляет на IP-адрес Алисы. 

* Пакет Боба проходит несколько промежуточных серверов, которые направляют его к Алисе.

* Сервер Алисы получает пакет Боба по своему IP 

* Сервер Алисы считывает порт назначения пакета. Грубо говоря, IP-адрес, это адрес дома. Порт - это квартира. Пакет передается в веб-сервер.

* Веб-сервер получает запрос на какой-то файл и находит его. Файл связывается в новый пакет и через маршрутиризатор отправляется Бобу. Ура! У нас есть интернет. 

Обратите внимание, что в этой схема браузер использовался ... нигде. Браузер - это относительно недавнее изобретение в истории интернета. Браузер - это довольно полезное приложение для создания пакетов информации. 


## 4. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его. Если у вас не установлен этот модуль, то придётся напрячься и установить, прописав в терминале `pip3 install requests`.

In [1]:
import requests

Делаем запрос по ссылке `mainpage` и получаем ответ на свой запрос!

In [2]:
main_page = 'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=1&room1=1&room2=1'
response = requests.get(main_page)
response

<Response [200]>

Благословенный 200 ответ - соединение установлено и данные получены, всё чудесно! Если попытаться перейти на несуществующую страницу, то можно получить, например, знаменитую ошибку 404, которая говорит о том, что клиент (Боб) был в состоянии общаться с сервером (Алисой), но не может найти данные согласно запросу.  

In [3]:
requests.get('https://www.cian.ru/jhgrgoiogr')

<Response [404]>

Вернёмся к успешному запросу и посмотрим что же такого мы там получили...

In [4]:
html = response.content
html

b'\n\n<!DOCTYPE html>\n<html class="no-touch" >\n    <head>\n        \n\n<!-- Google Analytics Content Experiment code -->\n<script>function utmx_section(){}function utmx(){}(function(){var\nk=\'57940811-35\',d=document,l=d.location,c=d.cookie;\nif(l.search.indexOf(\'utm_expid=\'+k)>0)return;\nfunction f(n){if(c){var i=c.indexOf(n+\'=\');if(i>-1){var j=c.\nindexOf(\';\',i);return escape(c.substring(i+n.length+1,j<0?c.\nlength:j))}}}var x=f(\'__utmx\'),xx=f(\'__utmxx\'),h=l.hash;d.write(\n\'<sc\'+\'ript src="\'+\'http\'+(l.protocol==\'https:\'?\'s://ssl\':\n\'://www\')+\'.google-analytics.com/ga_exp.js?\'+\'utmxkey=\'+k+\n\'&utmx=\'+(x?x:\'\')+\'&utmxx=\'+(xx?xx:\'\')+\'&utmxtime=\'+new Date().\nvalueOf()+(h?\'&utmxhash=\'+escape(h.substr(1)):\'\')+\n\'" type="text/javascript" charset="utf-8"><\\/sc\'+\'ript>\')})();\n</script><script>utmx(\'url\',\'A/B\');</script>\n<!-- End of Google Analytics Content Experiment code -->\n\n\n        \n    \n            <meta http-equiv="Content-Type"

Можно ли работать с этим ужасом? Ну... на самом деле да, но лучше не пробовать. Давайте сделаем всё красиво и используем для этого красивый суп! 

## 5. Красивый суп

<img align="center" src="https://www.crummy.com/software/BeautifulSoup/10.1.jpg" height="200" width="200"> 

Пакет **[`bs4`](https://www.crummy.com/software/BeautifulSoup/)**, a.k.a **BeautifulSoup** был назван в честь стишка про красивый суп из Алисы в стране чудес. В чём был смысл этого стишка, я не помню. Однако я помню зачем нужен этот пакет.

Красивый суп. Эта совершенно волшебная библиотека, которая из сырого и необработанного HTML (или XML) кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что вам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Так что нам нужен пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть однажды, а потом будет проще.

In [13]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [14]:
soup = BeautifulSoup(html,'html.parser') # В опции также можно указать lxml, 
                                         # если предварительно установить одноименный пакет
soup


<!DOCTYPE html>

<html class="no-touch">
<head>
<!-- Google Analytics Content Experiment code -->
<script>function utmx_section(){}function utmx(){}(function(){var
k='57940811-35',d=document,l=d.location,c=d.cookie;
if(l.search.indexOf('utm_expid='+k)>0)return;
function f(n){if(c){var i=c.indexOf(n+'=');if(i>-1){var j=c.
indexOf(';',i);return escape(c.substring(i+n.length+1,j<0?c.
length:j))}}}var x=f('__utmx'),xx=f('__utmxx'),h=l.hash;d.write(
'<sc'+'ript src="'+'http'+(l.protocol=='https:'?'s://ssl':
'://www')+'.google-analytics.com/ga_exp.js?'+'utmxkey='+k+
'&utmx='+(x?x:'')+'&utmxx='+(xx?xx:'')+'&utmxtime='+new Date().
valueOf()+(h?'&utmxhash='+escape(h.substr(1)):'')+
'" type="text/javascript" charset="utf-8"><\/sc'+'ript>')})();
</script><script>utmx('url','A/B');</script>
<!-- End of Google Analytics Content Experiment code -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Eig23WXwvPZu3gp9EYVreR61rG4TX1bHQDEG0Di5ZSE" name="google-site-verif

Что же лежит в переменной `soup`? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее это не так. Теперь мы можем свободно бродить по HTML-дереву страницы, искать детей, родителей и вытаскивать их! 

Например, можно бродить по вершинам, указывая путь из тегов.

In [17]:
soup.html.head.title

<title>
                Купить 1, 2-комнатную квартиру в Москве — 95 088 объявлений
            </title>

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [20]:
soup.html.head.title.text

'\n                Купить 1, 2-комнатную квартиру в Москве — 95 088 объявлений\n            '

Более того, зная адрес элемента, мы сразу можем найти его!

In [23]:
obj = soup.find('div', attrs = {'class':"serp-item__price-col"})
obj

<div class="serp-item__price-col">
<div class="serp-item__solid">
        8,99 млн. руб.
    </div>
<div class="serp-item__prop">
        
            
                178 728 руб.
            
            за м<sup>2</sup>
</div>
<div class="serp-item__prop">
            свободная
        </div>
<div class="objects_item_ipo objects_item_ipo_serp3">
<!--AdFox START-->
<!--cian-->
<!--Площадка: Cian.ru / Результаты поиска / Ипотека в поиске-->
<!--Категория: <не задана>-->
<!--Тип баннера: (cian) Ипотека в поиске-->
<!-- ________________________AdFox Asynchronous code START__________________________ -->
<div id="adfox_serp_item_mortgage_628128434"></div>
<script type="text/javascript">
  setTimeout(function() {
    if (typeof(pr) == 'undefined') { var pr = Math.floor(Math.random() * 4294967295) + 1; }
    if (typeof(document.referrer) != 'undefined') {
      if (typeof(afReferrer) == 'undefined') {
        afReferrer = encodeURIComponent(document.referrer);
      }
    } else {
      afR

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! 

In [24]:
obj2 = obj.find('div', attrs = {'class':"serp-item__solid"})
obj2

<div class="serp-item__solid">
        8,99 млн. руб.
    </div>

In [25]:
obj2.text

'\n        8,99 млн. руб.\n    '

Эх! Удалить бы ещё значки `\n`, которые означают смену строки и лишние пробелы... Сказано - сделано! Лишние пробелы убирает метод `strip`! 

In [26]:
obj2.text.strip()

'8,99 млн. руб.'

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они str. 

In [29]:
type(obj2.text.strip())

str

Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`. На выход будет выдан список!

In [49]:
dirty_prices = soup.findAll('div', attrs = {'class':"serp-item__price-col"})
dirty_prices[0].findAll('div',{'class':"serp-item__solid"})[0]

<div class="serp-item__solid">
        8,99 млн. руб.
    </div>

Повторим поиск. Сделаем это для каждого элемента из нашего нового списка...

In [51]:
clean_prices = [item.find('div', attrs = {'class':"serp-item__solid"}).text.strip() for item in dirty_prices]
clean_prices[:10]

['8,99 млн. руб.',
 '5,6 млн. руб.',
 '7,9 млн. руб.',
 '1,39 млн. руб.',
 '1,79 млн. руб.',
 '1,8 млн. руб.',
 '1,85 млн. руб.',
 '1,86 млн. руб.',
 '1,95 млн. руб.',
 '1,99 млн. руб.']

Собственно говоря, это всё. Полученные нами данные оказались немного грязноватыми. Везде есть слово `руб`, где-то стоимости указаны в тысячах, а где-то в миллионах...  Пока что бросим их в таком виде. Мало ли среди большого массива, который мы намерены скачать, есть ещё какие-то неточности. Лучше произвести очистку после выкачивания.  Оформим всё это дело в виде функции, чтобы прям красиво было...

In [52]:
def FlatPriceFinder(soup):
    dirty_prices = soup.findAll('div', attrs = {'class':"serp-item__price-col"})
    clean_prices = [item.find('div', attrs = {'class':"serp-item__solid"}).text.strip() for item in dirty_prices]
    return(clean_prices)

print(FlatPriceFinder(soup))

['8,99 млн. руб.', '5,6 млн. руб.', '7,9 млн. руб.', '1,39 млн. руб.', '1,79 млн. руб.', '1,8 млн. руб.', '1,85 млн. руб.', '1,86 млн. руб.', '1,95 млн. руб.', '1,99 млн. руб.', '1,99 млн. руб.', '1,99 млн. руб.', '1,99 млн. руб.', '2,03 млн. руб.', '2,1 млн. руб.', '2,16 млн. руб.', '2,2 млн. руб.', '2,21 млн. руб.', '2,23 млн. руб.', '2,26 млн. руб.', '2,29 млн. руб.', '2,29 млн. руб.', '2,29 млн. руб.', '2,3 млн. руб.', '2,3 млн. руб.', '2,33 млн. руб.', '2,39 млн. руб.', '2,39 млн. руб.']


Попробуем скачать ещё что-нибудь таким способом! Например, метраж!

In [66]:
obj1 = soup.findAll('div',{'class':"serp-item__area-col"})
print(obj1)

[<div class="serp-item__area-col">
<div class="serp-item__solid">50 м<sup>2</sup></div>
<div class="serp-item__prop">
        кухня 10 м<sup>2</sup>
</div>
<div class="serp-item__prop">
        жилая 28 м<sup>2</sup>
</div>
</div>, <div class="serp-item__area-col">
<div class="serp-item__solid">40 м<sup>2</sup></div>
<div class="serp-item__prop">
        кухня 9 м<sup>2</sup>
</div>
<div class="serp-item__prop">
        жилая 21 м<sup>2</sup>
</div>
</div>, <div class="serp-item__area-col">
<div class="serp-item__solid">56 м<sup>2</sup></div>
<div class="serp-item__prop">
        кухня 9 м<sup>2</sup>
</div>
<div class="serp-item__prop">
        жилая 40 м<sup>2</sup>
</div>
</div>, <div class="serp-item__area-col">
<div class="serp-item__solid">15 м<sup>2</sup></div>
<div class="serp-item__prop">
        кухня 4 м<sup>2</sup>
</div>
<div class="serp-item__prop">
        жилая 11 м<sup>2</sup>
</div>
</div>, <div class="serp-item__area-col">
<div class="serp-item__solid">21 м<sup>2</su

In [69]:
obj2  = [item.findAll('div',{'class':"serp-item__solid"})[0].text for item in obj1]
print(obj2)

['50\xa0м2', '40\xa0м2', '56\xa0м2', '15\xa0м2', '21\xa0м2', '35\xa0м2', '34\xa0м2', '27\xa0м2', '34\xa0м2', '26\xa0м2', '21\xa0м2', '25\xa0м2', '29\xa0м2', '20\xa0м2', '35\xa0м2', '22\xa0м2', '32\xa0м2', '21\xa0м2', '21\xa0м2', '23\xa0м2', '27\xa0м2', '27\xa0м2', '23\xa0м2', '22\xa0м2', '22\xa0м2', '24\xa0м2', '25\xa0м2', '26\xa0м2']


В принципе, по большей части, это всё. Все остальные кусочки с ЦИАН забираются аналогичным образом. Пишем ещё с десяток подобных функций! И всё хорошо. Однако, если нажать на кнопку "Подробнее", мы увидим чудо-чудное, диво-дивное! Дополнительную информацию! 


<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/3.png" height="700" width="700">


## 6. Качаем всё, что плохо лежит!

### 6.1 Список из хрефов

Для начала соберём все хрефы на квартиры! Сначала как всегода с помощью красивого супа мы найдём ссылки на все хрефы.

In [70]:
hrefs = soup.findAll('div', attrs = {'class':"serp-item__content__bottom__left"})
# убедимся, что мы не насобирали лишнего
print(len(hrefs))
hrefs[0]

28


<div class="serp-item__content__bottom__left">
<span class="serp-item__print-info">
					Александр Григорьев, +7 962 961-72-32
				</span>
<a class="serp-item__card-link link" href="https://www.cian.ru/sale/flat/151747032/" target="_blank">Подробнее</a>
</div>

Перейдём по вытащенному кусочку с хрефом в часть с тэгом `<a>`, а после вытащим из неё атрибут `href`. Так мы ещё не делали, но так тоже можно... 

In [76]:
hrefs[0].a.attrs['href']

'https://www.cian.ru/sale/flat/151747032/'

Делаем по аналогии со всеми хрефами!

In [77]:
clean_hrefs = [item.a.attrs['href'] for item in hrefs]
print(clean_hrefs)

['https://www.cian.ru/sale/flat/151747032/', 'https://www.cian.ru/sale/flat/156180593/', 'https://www.cian.ru/sale/flat/149788313/', 'https://www.cian.ru/sale/flat/155214919/', 'https://www.cian.ru/sale/flat/156363994/', 'https://www.cian.ru/sale/flat/154412243/', 'https://www.cian.ru/sale/flat/154243990/', 'https://www.cian.ru/sale/flat/155993966/', 'https://www.cian.ru/sale/flat/153192250/', 'https://www.cian.ru/sale/flat/155771032/', 'https://www.cian.ru/sale/flat/154716281/', 'https://www.cian.ru/sale/flat/155912953/', 'https://www.cian.ru/sale/flat/155251001/', 'https://www.cian.ru/sale/flat/155835372/', 'https://www.cian.ru/sale/flat/153758143/', 'https://www.cian.ru/sale/flat/151729871/', 'https://www.cian.ru/sale/flat/152087214/', 'https://www.cian.ru/sale/flat/151730037/', 'https://www.cian.ru/sale/flat/154252525/', 'https://www.cian.ru/sale/flat/151729984/', 'https://www.cian.ru/sale/flat/155541671/', 'https://www.cian.ru/sale/flat/156407537/', 'https://www.cian.ru/sale/flat/

Нам надо будет путешествовать по огромному длинному списку из квартир, который простирается очень далеко вглубь ЦИАН. Пробираться вглубь сайта можно тыкая на циферки внизу старницы 

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/4.png" height="900" width="900">

Как только мы нажимаем на одну из цифр, меняется ссылка.

    `https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&
    p=3&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1`
   
    `https://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=8&engine_version=2&offer_type=flat&
     p=10&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1`
     
Обратите внимание на начало второй строки. Когда мы находились на третьей строке, там стояла цифра 3. Как толкьо мы переходим на десятую страницу, цифра меняется на 10. Сама по себе ссылка на квартиры формируется в заисимости от того какие огранчения мы накладываем на квартиры в поиске. Каждое новое ограничение вырождается в новый кусочек ссылки. Например, если наложить ограничения на районы Москвы, то ссылка будет выглядеть вот так: 

    `https://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=1&district%5B1%5D=4&
    district%5B2%5D=5&district%5B3%5D=6&district%5B4%5D=7&district%5B5%5D=8&district%5B6%5D=9&
    district%5B7%5D=10&district%5B8%5D=11&engine_version=2&offer_type=flat&
    p=10&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1`

Пройдёмся по всем существующим страницам ЦИАН и заберём себе все ссылки на квартиры! 

Каждую ссылку будем формировать следующим образом:

In [80]:
page_part_1 = "https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p="
page_part_2 = "&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"

page = page_part_1 + str(42) + page_part_2  # Мы окжемся на 42 странице
page

'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p=42&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'

Всего на каждой странице находится 28 ссылок. Вытаскиваем необходимое нам число квартир.

In [81]:
page_part_1 = "https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p="
page_part_2 = "&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"

all_hrefs = [ ]  # Тут мы будем хранить наши хрефы!

for i in range(1,11): # Тут регулируем количество квартир
    # Делаем ссылку!
    page = page_part_1 + str(i) + page_part_2
    
    # Загружаем страницу
    response = requests.get(page)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # Забираем себе хрефы и очищаем их
    hrefs = soup.findAll('div', attrs = {'class':"serp-item__content__bottom__left"})
    clean_hrefs = [item.a.attrs['href'] for item in hrefs]
    all_hrefs.extend(clean_hrefs)

In [84]:
len(all_hrefs)

280

Ого сколько хрефов! Пора качать данные по каждому из них! 

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/1.jpg" height="500" width="500">

### 6.2 Качаем данные по конкретному хрефу

Перейдём по какому-нибудь одному хрефу и заберём всю информацию о квартире! Вообще всю. 

In [88]:
response = requests.get(clean_hrefs[0])
html = response.content
soup = BeautifulSoup(html,'html.parser')

Собираем информацию...

In [89]:
price = soup.findAll('div', attrs = {'class':"object_descr_price"})[0].text.strip()
price

'27 062 000 руб.'

In [102]:
soup.findAll('p', attrs = {'class':"objects_item_metro_prg"})[0].a.text

'Шаболовская,'

In [105]:
metro_infa = soup.findAll('p', attrs = {'class':"objects_item_metro_prg"})[0]
metro_infa.find('span',{'class':'object_item_metro_comment'}).text.strip()

'10\n                мин.\n                \n                    пешком'

Дело осталось за малым. Достать основную информацию о квартире... Посмотрим где она лежит на странице через inspect где она лежит и заметим, что она лежит в таблице! 

"Хммм...", - скажем мы и начнём эту таблицу разгребать. Будем делать это хардкорно, по тэгам. Как настоящие мужики! 

In [108]:
table = soup.findAll('table',attrs = {'class':"object_descr_props flat sale"})[0]
table

<table class="object_descr_props flat sale" style="float:left">
<tr>
<th style="padding-top:5px;">Общая информация:</th>
</tr>
<tr>
<th>Этаж:</th>
<td>
                2 / 18
                
            </td>
</tr>
<tr>
<th>Тип дома:</th>
<td>
            вторичка, 
            монолитный
        </td>
</tr>
<tr>
<th>Тип продажи:</th>
<td>свободная</td>
</tr>
<tr>
<th>Общая площадь:</th>
<td>
<i class="object_descr_details_color"></i>112 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь комнат:</th>
<td>
<i class="object_descr_details_color"></i><!--
-->15-25-30 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Жилая площадь:</th>
<td>
<i class="object_descr_details_color" style="background-color:#90C090"></i>70 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь кухни:</th>
<td>
<i class="object_descr_details_color" style="background-color:#C09090"></i>16 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Санузел:</th>
<td><i class="object_descr_details_color" style="background-color:#90C0C0"></i>нет
        </td>
</tr>
<tr>
<th>

Вытаскиваем все строки

In [113]:
rows = table.findAll('tr')[1:]  # В нулевой строке идёт фраза "Общая информация:" Забьём на неё! 
rows[0]  

<tr>
<th>Этаж:</th>
<td>
                2 / 18
                
            </td>
</tr>

Находим конкретные элементы!

In [118]:
rows[8].findAll('th')[0].text.strip()

'Балкон:'

In [119]:
rows[8].findAll('td')[0].text.strip()

'1 балк.'

Такимии незатейливыми действиями вытащили отдельный объект из отдельной строки. А теперь всё сразу в виде двух циклов:

In [120]:
rows = table.find_all('tr')[1:]  # Вытащили все строки
data = [ ]  # Задали пустой список, в который мы будем сохранять строки 
for row in rows:
    cols = [row.findAll('th')[0].text.strip(),row.findAll('td')[0].text.strip()] # очищаем каждую строку! 
    data.append(cols) # запоминаем
data 

[['Этаж:', '2\xa0/\xa018'],
 ['Тип дома:', 'вторичка, \n            монолитный'],
 ['Тип продажи:', 'свободная'],
 ['Общая площадь:', '112\xa0м2'],
 ['Площадь комнат:', '15-25-30\xa0м2'],
 ['Жилая площадь:', '70\xa0м2'],
 ['Площадь кухни:', '16\xa0м2'],
 ['Санузел:', 'нет'],
 ['Балкон:', '1 балк.'],
 ['Лифт:', '1 пасс. + 1 груз.'],
 ['Телефон:', 'нет'],
 ['Вид из окна:', 'двор'],
 ['Ремонт:', 'отсутствует']]

Чуток всё это облагородим. Подгрузим пакет, который носит гордое название pandas! Подробнее про этот пакет мы поговорим немного позже. Если быть кратким, то pandas используется для работы с таблицами. Превратим наш список из списков в красивую удобную табличку. 

In [125]:
import pandas as pd
data = pd.DataFrame(data)
data.head()

,0,1
0,Этаж:,2 / 18
1,Тип дома:,"вторичка, \n монолитный"
2,Тип продажи:,свободная
3,Общая площадь:,112 м2
4,Площадь комнат:,15-25-30 м2


Отлично! Почти вся информация в сборе. Не хватает только одного: географических координат. Они понадобятся нам для вычисления различных расстояний. На рассматриваемой нами странице есть карта. Если мы можем открыть карту, на которую нанесён дом, значит где-то в HTML страницы спрятаны географические координаты! Попробуем отыскать их. 

Для этого откроем [Яндекс - карты](https://yandex.ru/maps) и вобьём адрес любой недвижимости с ЦИАН. Скопируем одну из координат объекта. Откроем код ЦИАНовской странички и воспользуемся поиском по ней. Опа! Мы нашли нужный нам тэг...

In [126]:
coordinates = soup.findAll('div',{'class':"object-descr__map-tabs__content js-object_descr__panorama"})[0]
coordinates

<div class="object-descr__map-tabs__content js-object_descr__panorama">
<panorama ns="ymapspano" point="[37.606963,55.710677]"></panorama>
</div>

In [127]:
coordinates.panorama.attrs['point']  # перешли по тэгу, после по атрубуту и координаты у нас в кармане.

'[37.606963,55.710677]'

Пришло время соеденить всё это в одно целое. Функция для выкачки данных по одной квартире будет выглядеть следующим образом: 

In [131]:
def One_Flat_Downloader(href):
    """
    Функция производит выкачку по одной ЦИАНовской ссылке 
    всей существующей информации о квартире.
    Ввод: ссылка на описание квартиры
    Вывод: словарь с информацией о квартире    
    """
    
    data = { }  # Задали пустой словарь, в который мы будем сохранять данные
    
    # Подгружаем страничку с информацией по квартире
    response = requests.get(href)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # Вытаскиваем цену на квартиру
    price = soup.findAll('div', attrs = {'class':"object_descr_price"})[0].text.strip()
    data['Цена'] = price
    
    # Вытаскиваем метро
    station = soup.findAll('p', attrs = {'class':"objects_item_metro_prg"})[0].a.text
    data['Метро'] = station
    
    # Вытаскиваем расстояние до метро то ли пешком то ли на машине...
    Do_metro = soup.findAll('p', attrs = {'class':"objects_item_metro_prg"})[0]
    Do_metro = Do_metro.find('span',{'class':'object_item_metro_comment'}).text.strip()
    data['До метро'] = Do_metro
    
    # Вытаскиваем координаты квартиры
    coordinates = soup.findAll('div',{'class':"object-descr__map-tabs__content js-object_descr__panorama"})[0]
    data['Координаты'] = coordinates.panorama.attrs['point']
    
    # Вытаскиваем всё остальное
    table = soup.findAll('table',attrs = {'class':"object_descr_props flat sale"})[0]
    rows = table.find_all('tr')[1:]  # Вытащили все строки
    for row in rows:
        cols = [row.findAll('th')[0].text.strip(),row.findAll('td')[0].text.strip()] # очищаем каждую строку! 
        data[cols[0]] = cols[1] # запоминаем
    return(data) 

Попробуем применить эту функцию к нескольким разным квартирам. Функция каждый раз будет возвращать нам табличку, которая относится к данной квартире. Каждый раз табличка будет содержать разное количество строк, потому что информация о каждой квартире в своём роде уникальна...

In [134]:
dic = One_Flat_Downloader(clean_hrefs[0])
pd.DataFrame.from_dict(dic,orient='index')

,0
Этаж:,2 / 18
Тип продажи:,свободная
До метро,10\n мин.\n \n ...
Санузел:,нет
Балкон:,1 балк.
Общая площадь:,112 м2
Площадь кухни:,16 м2
Тип дома:,"вторичка, \n монолитный"
Телефон:,нет
Вид из окна:,двор


In [135]:
dic = One_Flat_Downloader(clean_hrefs[-1])
pd.DataFrame.from_dict(dic,orient='index')

,0
Этаж:,7 / 24
Тип продажи:,свободная
До метро,15\n мин.\n \n ...
Балкон:,нет
Общая площадь:,"22,1 м2"
Площадь кухни:,"3,4 м2"
Тип дома:,"новостройка, \n кирпично-монолитный"
Сдача ГК:,1 кв. 2017 года
Совмещенных санузлов:,1
Вид из окна:,двор


### 6.3 Агрегирование 

Итак, что у нас есть? 

* Огромный список из хрефов, которые надо пройти
* Функция, которая по каждому хрефу собирает для нас нужные данные

Что мы хотим? 
* Мы хотим, чтобы все эти данные оказались в одной большой таблице. Нужно написать функцию, которая будет соединять словари в единое целое. При этом, если в однои из словарей нет какого-нибудь ключа, в итоговом словаре он должен заполняться как NA. 

In [136]:
df1 = pd.DataFrame.from_dict(One_Flat_Downloader(all_hrefs[0]),orient='index')
df2 = pd.DataFrame.from_dict(One_Flat_Downloader(all_hrefs[1]),orient='index')

In [137]:
print(df1.shape,df2.shape) # посмотрим размерности получившихся таблиц

(18, 1) (16, 1)


In [138]:
df1.head(3)

,0
Этаж:,11 / 13
Тип продажи:,переуступка
До метро,8\n мин.\n \n ...


In [141]:
df2.head(3)

,1
Этаж:,2 / 4
Тип продажи:,свободная
До метро,10\n мин.\n \n ...


Обратим внимание, что у наблюдений одинаковые номера! Ноль и ... ноль. Надо как-то решить эту проблему. Переименуем у второй таблицы номер наблюдения с нулевого на первый, а после объединим таблички с помощью команды `join`. Если номер таблицы не заменить, то команда не сработает.

In [142]:
df2.columns =[1]
df = df1.join(df2, how='outer') # Если поставить вместо outer - inner, то останутся только общие строки
                                # Если поставить left, то тольо строки из левой таблицы
                                # Если только rigth, то только строки из правой таблицы. 
df.shape

(19, 2)

In [150]:
# Пример на команду join: 
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                    'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])

right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                     'D': ['D0', 'D2', 'D3']},
                     index=['K0', 'K2', 'K3'])
left.join(right,how = 'right')

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [151]:
# Посмотрим на нашу таблицу! 
df.head()

,0,1
Балкон:,1 лодж.,–
Вид из окна:,двор и улица,–
До метро,8\n мин.\n \n ...,10\n мин.\n \n ...
Жилая площадь:,51 м2,–
Координаты,"[37.430920,55.821947]","[37.616863,55.768132]"


В принципе, всё готово. Можно объединять таблицы по всем хрефам. В атаку! 

In [152]:
# В эту табличку будем собирать данные
df = pd.DataFrame( )
k = 0 # Это номера наблюдений
for item in all_hrefs:
    k = k + 1
    # грузим новое наблюдение
    df1 = pd.DataFrame.from_dict(One_Flat_Downloader(item),orient='index')
    # присваиваем этому наблюдению номер
    df1.columns =[k]
    # закидываем его в итоговую таблицу
    df = df.join(df1, how='outer')
df.head()

IndexError: list index out of range

ОШИБКА?! Но я же вроде бы всё писал правильно...  А ещё я запускал код позавчера и он ... работал! А сейчас всё сломалось и не работает... Я в шоке! Что мне делать?! БЛИИИИИИИН


<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/5.jpg" height="700" width="700">


Попытаемся понять где именно произошёл сбой.

In [153]:
l = 0
for item in all_hrefs:
    One_Flat_Downloader(item)
    l = l + 1

IndexError: list index out of range

In [154]:
l

2

In [155]:
all_hrefs[2]

'https://www.cian.ru/sale/flat/154707072/'

Сбой произошёл на шестой ссылке! Переходим по ней, смотрим что там происходит иииии... там Новая Москва, а в Новой Москве нет метрою Именно поэтому скрипт не может найти его и пишет нам, что `object has no attribute 'text'`.

Что же нам делать? Конечно же исключения! В python существует специальная конструкция, которая называется try - except. Посмотрим на то как она работает: 

```python
try:
    something
except Name_of_some_error:
    do_something_else
```

> позволяет в случае, если произошла ошибка типа Name_of_some_error не заканчивать программу со словами «Все пропало! Ошибка!», а тут же передать управление блоку do_something_else, который что-нибудь сделает.

In [156]:
from math import log
a = [1,2,3,4,-5,-7,9,10,0,12,2]
b = [ ]
for atem in a:
    b.append(log(atem))
print(b)

ValueError: math domain error

In [157]:
b = [ ]
for atem in a:
    try:
        b.append(log(atem))
    except Exception:
        b.append('NA')
print(b)

[0.0, 0.6931471805599453, 1.0986122886681098, 1.3862943611198906, 'NA', 'NA', 2.1972245773362196, 2.302585092994046, 'NA', 2.4849066497880004, 0.6931471805599453]


Видите, что произошло. Когда логарифм не мог вычислиться, в вектор записывалось NA. Про исключения можно почитать [на просорах интернета](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html). В данном примере мы наложили очень сильное исключение, которое работает при возникновении абсолютно любой неадекватной ситуации в коде. Это исключение при необходимость можно ослабить, заменив на другое.

Пропишем в функции, которая выкачивает странице в графе, которая отвечает за станцию метро такое исключение. Итоговый код для скачки ЦИАН будет иметь следующий вид: 

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Первая часть кода. Вытаскиваем список хрефов.

page_part_1 = "https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p="
page_part_2 = "&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"

all_hrefs = [ ]  # Тут мы будем хранить наши хрефы!

for i in range(1,200): # Тут регулируем количество квартир (на одной странице их 28)
    # Делаем ссылку!
    page = page_part_1 + str(i) + page_part_2
    
    # Загружаем страницу
    response = requests.get(page)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # Забираем себе хрефы и очищаем их
    hrefs = soup.findAll('div', attrs = {'class':"serp-item__content__bottom__left"})
    clean_hrefs = [item.a.attrs['href'] for item in hrefs]
    all_hrefs.extend(clean_hrefs)
    print('Качаю хреф номер ',i)
    

### Вторая часть кода. Функция, которая вытаскивает наблюдения по одной квартирке.    
    
def One_Flat_Downloader(href):
    """
    Функция производит выкачку по одной ЦИАНовской ссылке 
    всей существующей информации о квартире.
    Ввод: ссылка на описание квартиры
    Вывод: словарь с информацией о квартире    
    """
    
    data = { }  # Задали пустой словарь, в который мы будем сохранять данные
    
    # Подгружаем страничку с информацией по квартире
    response = requests.get(href)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # Вытаскиваем цену на квартиру
    price = soup.findAll('div', attrs = {'class':"object_descr_price"})[0].text.strip()
    data['Цена'] = price
    
    # Вытаскиваем метро
    try:
        station = soup.findAll('p', attrs = {'class':"objects_item_metro_prg"})[0].a.text
        data['Метро'] = station
    except Exception:
        data['Метро'] = "NA"
    
    # Вытаскиваем расстояние до метро то ли пешком то ли на машине...
    try:
        Do_metro = soup.findAll('p', attrs = {'class':"objects_item_metro_prg"})[0]
        Do_metro = Do_metro.find('span',{'class':'object_item_metro_comment'}).text.strip()
        data['До метро'] = Do_metro
    except Exception:
        data['До метро'] = "NA"
    
    # Вытаскиваем координаты квартиры
    coordinates = soup.findAll('div',{'class':"object-descr__map-tabs__content js-object_descr__panorama"})[0]
    data['Координаты'] = coordinates.panorama.attrs['point']
    
    # Вытаскиваем всё остальное
    table = soup.findAll('table',attrs = {'class':"object_descr_props flat sale"})[0]
    rows = table.find_all('tr')[1:]  # Вытащили все строки
    for row in rows:
        cols = [row.findAll('th')[0].text.strip(),row.findAll('td')[0].text.strip()] # очищаем каждую строку! 
        data[cols[0]] = cols[1] # запоминаем
    return(data) 


### Третья часть кода. Собираем данные по всем хрефам и объединяем их в огромную таблицу.

# В эту табличку будем собирать данные
df = pd.DataFrame( )
k = 0 # Это номера наблюдений
n = len(all_hrefs)
for item in all_hrefs:
    k = k + 1
    # грузим новое наблюдение
    df1 = pd.DataFrame.from_dict(One_Flat_Downloader(item),orient='index')
    # присваиваем этому наблюдению номер
    df1.columns =[k]
    # закидываем его в итоговую таблицу
    df = df.join(df1, how='outer')
    # Выдавать информацию о том, что сделана каждая десятая итерация! 
    if k%10 == 0:
        print('скачал ', k, ' итераций из', n)


Качаю хреф номер  1
Качаю хреф номер  2
Качаю хреф номер  3
Качаю хреф номер  4
Качаю хреф номер  5
Качаю хреф номер  6
Качаю хреф номер  7
Качаю хреф номер  8
Качаю хреф номер  9
Качаю хреф номер  10
Качаю хреф номер  11
Качаю хреф номер  12
Качаю хреф номер  13
Качаю хреф номер  14
Качаю хреф номер  15
Качаю хреф номер  16
Качаю хреф номер  17
Качаю хреф номер  18
Качаю хреф номер  19
Качаю хреф номер  20
Качаю хреф номер  21
Качаю хреф номер  22
Качаю хреф номер  23
Качаю хреф номер  24
Качаю хреф номер  25
Качаю хреф номер  26
Качаю хреф номер  27
Качаю хреф номер  28
Качаю хреф номер  29
Качаю хреф номер  30
Качаю хреф номер  31
Качаю хреф номер  32
Качаю хреф номер  33
Качаю хреф номер  34
Качаю хреф номер  35
Качаю хреф номер  36
Качаю хреф номер  37
Качаю хреф номер  38
Качаю хреф номер  39
Качаю хреф номер  40
Качаю хреф номер  41
Качаю хреф номер  42
Качаю хреф номер  43
Качаю хреф номер  44
Качаю хреф номер  45
Качаю хреф номер  46
Качаю хреф номер  47
Качаю хреф номер  48
К

In [6]:
df = df.T  # Для удобства транспонируем таблицу
df.head( )

,Балкон:,Вид из окна:,Высота потолков:,Год постройки:,До метро,Жилая площадь:,Координаты,Лифт:,Метро,Общая площадь:,...,Ремонт:,Санузел:,Сдача ГК:,Совмещенных санузлов:,Статус помещения:,Телефон:,Тип дома:,Тип продажи:,Цена,Этаж:
1,–,двор,"3,50 м",2012,4\n мин.\n \n ...,"121,7 м2","[37.653703,55.744738]",2 пасс. + 2 груз.,"Таганская,","194,7 м2",...,отсутствует,NaN,NaN,3,NaN,NaN,вторичка,свободная,123 000 000 руб.,7 / 9
2,1 балк.,улица,NaN,NaN,10\n мин.\n \n ...,"30,1 м2","[37.414421,55.633661]",1 пасс. + 1 груз.,"Саларьево,","54,0 м2",...,NaN,NaN,3 кв. 2017 года,NaN,NaN,NaN,"новостройка, \n панельный дом",свободная,5 579 537 руб.,3 / 21
3,1 лодж.,улица,"2,64 м",1965,16\n мин.\n \n ...,17 м2,"[37.554987,55.871930]",1 пасс.,"Петровско-Разумовская,",33 м2,...,евроремонт,NaN,NaN,1,NaN,да,вторичка,свободная,5 200 000 руб.,5 / 9
4,–,–,NaN,NaN,25\n мин.\n \n ...,26 м2,"[37.322593,55.648382]",–,"Киевская,",29 м2,...,отсутствует,NaN,NaN,1,NaN,NaN,вторичка,свободная,1 440 000 руб.,3 / 3
5,–,–,"2,64 м",1972,16\n мин.\n \n ...,10 м2,"[37.617986,55.606690]",–,"Пражская,",12 м2,...,отсутствует,NaN,NaN,1,NaN,NaN,вторичка,свободная,1 540 000 руб.,1 / 9


In [7]:
df.shape

(5572, 25)

Сохраняем данные в формате csv... 

In [8]:
pd.DataFrame.to_csv(df,'CIAN_data.csv')